# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key as gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv('weather_data.csv')
weather_data

,Unnamed: 0,City,Cloud Coverage (%),Country,Date,Humidity (%),Temperature (F),Latitude,Longitude,Wind Speed
0,0,albany,20,US,1586647210,28,46.99,42.60,-73.97,19.46
1,1,mataura,93,NZ,1586647259,62,62.01,-46.19,168.86,3.00
2,2,kodiak,90,US,1586647259,80,41.00,57.79,-152.41,18.34
3,3,evensk,100,RU,1586647259,88,13.01,61.95,159.23,2.46
4,4,puerto ayora,75,EC,1586647259,70,84.20,-0.74,-90.35,10.29
...,...,...,...,...,...,...,...,...,...,...
579,579,kibungo,40,RW,1586647397,88,68.00,-2.16,30.54,2.24
580,580,istok,24,XK,1586647397,65,51.01,42.78,20.49,3.36
581,581,katobu,100,ID,1586647397,93,75.76,-4.94,122.53,1.70
582,582,kulhudhuffushi,41,MV,1586647266,64,85.68,6.62,73.07,6.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=gkey)
locations = weather_data[['Latitude', 'Longitude']]
locations
humidity = weather_data["Humidity (%)"]
humidity
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, humidity))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = weather_data.loc[(weather_data["Temperature (F)"] > 70) & (weather_data["Temperature (F)"] < 80) & (weather_data["Cloud Coverage (%)"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head(10)

,Unnamed: 0,City,Cloud Coverage (%),Country,Date,Humidity (%),Temperature (F),Latitude,Longitude,Wind Speed
0,198,farafenni,0,GM,1586647308,60,73.83,13.57,-15.60,8.25
1,224,pisco,0,PE,1586647087,30,75.00,-13.70,-76.22,6.93
2,315,puerto suarez,0,BO,1586647334,62,72.55,-18.95,-57.80,2.73
3,394,tekkali,0,IN,1586647352,79,77.99,18.62,84.23,6.51
4,422,umaria,0,IN,1586647358,21,74.66,23.53,80.83,3.29
5,440,concordia,0,AR,1586647362,56,71.60,-31.39,-58.02,5.82
6,465,mawlaik,0,MM,1586647368,44,70.81,23.63,94.42,1.25
7,470,ballina,0,AU,1586647369,49,71.60,-28.87,153.57,13.87
8,483,nerang,0,AU,1586647372,41,78.01,-27.98,153.33,12.75
9,528,kavali,0,IN,1586647275,89,74.26,14.92,79.98,6.53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey,
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrowed_city_df["Hotel Name"] = hotel_df
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloud Coverage (%),Country,Date,Humidity (%),Temperature (F),Latitude,Longitude,Wind Speed,Hotel Name
0,198,farafenni,0,GM,1586647308,60,73.83,13.57,-15.60,8.25,Farafenni
1,224,pisco,0,PE,1586647087,30,75.00,-13.70,-76.22,6.93,Pisco
2,315,puerto suarez,0,BO,1586647334,62,72.55,-18.95,-57.80,2.73,Puerto Suárez
3,394,tekkali,0,IN,1586647352,79,77.99,18.62,84.23,6.51,Tekkali
4,422,umaria,0,IN,1586647358,21,74.66,23.53,80.83,3.29,Umaria


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]


In [46]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))